<a href="https://colab.research.google.com/github/Ribeirotmr/Traveling-Salesman-Problem-/blob/main/Quantum-TSP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install qiskit -U
!pip install qiskit_aer
!pip install qiskit-ibm-runtime

import qiskit
qiskit.__version__

!pip install matplotlib
!pip install pylatexenc

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 73.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.9/363.9 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.5/69.5 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136817 sha25

In [ ]:


import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_aer import AerSimulator
from itertools import permutations
from qiskit.visualization import plot_histogram



In [ ]:



def generate_tsp_graph(ncidades):
    G = nx.complete_graph(ncidades)
    for (u, v) in G.edges():
        peso = np.random.randint(1, 100)
        G[u][v]['weight'] = peso
    distances = np.zeros((ncidades, ncidades))
    for i in range(ncidades):
        for j in range(ncidades):
            if i != j:
                distances[i][j] = G[i][j]['weight']
    print("Arestas com pesos:")
    for (u, v, d) in G.edges(data=True):
        print(f"({u}, {v}) -> peso: {d['weight']}")
    print("\nMatriz de distâncias:")
    print(distances)
    return G, distances


def compute_path_cost(path, distances):
    cost = 0
    for i in range(len(path)-1):
        cost += distances[path[i]][path[i+1]]
    cost += distances[path[-1]][path[0]]
    return cost

def brute_force_tsp(distances):
    n = len(distances)
    best_cost = float('inf')
    best_order = None
    for perm in permutations(range(1, n)):
        path = [0] + list(perm)
        cost = compute_path_cost(path, distances)
        print(f"order = {path} Distance = {cost}")
        if cost < best_cost:
            best_cost = cost
            best_order = path
    print(f"\nBest order = {best_order} with total distance = {best_cost}")
    return best_order, best_cost

def create_qaoa_tsp_restriction(ncidades, gamma, beta, distances, penalty=1000):
    qr = QuantumRegister(ncidades**2)
    cr = ClassicalRegister(ncidades**2)
    qc = QuantumCircuit(qr, cr)


    for q in qr:
        qc.h(q)

    for i in range(ncidades):
        for j in range(ncidades):
            if i != j:
                idx = i * ncidades + j

                for k in range(j+1, ncidades):
                    idx2 = i * ncidades + k
                    qc.cx(qr[idx], qr[idx2])
                    qc.rz(2 * gamma * penalty, qr[idx2])
                    qc.cx(qr[idx], qr[idx2])

                for k in range(i+1, ncidades):
                    idx3 = k * ncidades + j
                    qc.cx(qr[idx], qr[idx3])
                    qc.rz(2 * gamma * penalty, qr[idx3])
                    qc.cx(qr[idx], qr[idx3])


    for i in range(ncidades):
        for j in range(ncidades):
            if i != j:
                idx = i * ncidades + j
                qc.rz(2 * gamma * distances[i][j], qr[idx])


    for q in qr:
        qc.rx(2 * beta, q)

    qc.measure(qr, cr)
    return qc


def interpret_bitstring(bitstring, ncidades):
    path = []
    bits = [int(b) for b in bitstring[::-1]]
    for i in range(ncidades):
        for j in range(ncidades):
            if bits[i * ncidades + j] == 1:
                path.append(j)
                break
    return path

ncidades = 5
G, distances = generate_tsp_graph(ncidades)
best_order, best_cost = brute_force_tsp(distances)


gamma = 0.5
beta = 0.5
penalty = 1000

qc = create_qaoa_tsp_restriction(ncidades, gamma, beta, distances, penalty)

backend = AerSimulator()
result = backend.run(qc, shots=1024).result()
counts = result.get_counts()


interpreted = {}
for bitstring, freq in counts.items():
    path = interpret_bitstring(bitstring, ncidades)
    if len(set(path)) == ncidades:
        cost = compute_path_cost(path, distances)
        interpreted[tuple(path)] = interpreted.get(tuple(path), 0) + freq

print("\nSoluções interpretadas:")
for path, freq in interpreted.items():
    print(f"Caminho: {path}, Freq: {freq}")

plot_histogram(counts)
plt.show()


Arestas com pesos:
(0, 1) -> peso: 59
(0, 2) -> peso: 37
(0, 3) -> peso: 94
(0, 4) -> peso: 27
(1, 2) -> peso: 74
(1, 3) -> peso: 1
(1, 4) -> peso: 95
(2, 3) -> peso: 30
(2, 4) -> peso: 40
(3, 4) -> peso: 25

Matriz de distâncias:
[[ 0. 59. 37. 94. 27.]
 [59.  0. 74.  1. 95.]
 [37. 74.  0. 30. 40.]
 [94.  1. 30.  0. 25.]
 [27. 95. 40. 25.  0.]]
order = [0, 1, 2, 3, 4] Distance = 215.0
order = [0, 1, 2, 4, 3] Distance = 292.0
order = [0, 1, 3, 2, 4] Distance = 157.0
order = [0, 1, 3, 4, 2] Distance = 162.0
order = [0, 1, 4, 2, 3] Distance = 318.0
order = [0, 1, 4, 3, 2] Distance = 246.0
order = [0, 2, 1, 3, 4] Distance = 164.0
order = [0, 2, 1, 4, 3] Distance = 325.0
order = [0, 2, 3, 1, 4] Distance = 190.0
order = [0, 2, 3, 4, 1] Distance = 246.0
order = [0, 2, 4, 1, 3] Distance = 267.0
order = [0, 2, 4, 3, 1] Distance = 162.0
order = [0, 3, 1, 2, 4] Distance = 236.0
order = [0, 3, 1, 4, 2] Distance = 267.0
order = [0, 3, 2, 1, 4] Distance = 320.0
order = [0, 3, 2, 4, 1] Distance = 318